# Лабораторная работа №2
## Феоктистова Эмма
## 2 курс, 3 группа, ФиКЛ

Обработаем сайт:

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://izvmor.ru/"

page = requests.get(url)
soup = BeautifulSoup(page.text)

In [3]:
urls_get = []

for i in range(0, 1200, 12): 
    # изначально тут должны быть range(0, 24168, 12), однако такое количество страниц новостей не обработается даже за ночь
    # поэтому условно я сделала обработку 100 страниц из 2015
    # чтобы обработать абсолютно все новости на портале, нужно соответственно подставить вместо 1200 число 24168
    new_url = url + 'novosti?start=' + str(i)
    urls_get.append(new_url)
    
len(urls_get)

100

Оставим только ссылки на новости:

In [4]:
urls = []

for url in urls_get:
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    for link in soup.find_all('a'):
        if link.get('href') != None:
            if '/novosti' in link.get('href'):
                urls.append(link.get('href'))
len(urls)

8372

Добавим к полученным ссылкам основную ссылку на сайт, чтобы получить полные ссылки:

In [5]:
full_urls = []

for u in urls:
    res = 'https://izvmor.ru' + u
    full_urls.append(res) 

full_urls

['https://izvmor.ru/novosti/natsproekty?layout=blog',
 'https://izvmor.ru/novosti/natsproekty?layout=blog',
 'https://izvmor.ru/novosti',
 'https://izvmor.ru/novosti/proisshestviya',
 'https://izvmor.ru/novosti/obshchestvo',
 'https://izvmor.ru/novosti/politika',
 'https://izvmor.ru/novosti/ekonomika',
 'https://izvmor.ru/novosti/sport',
 'https://izvmor.ru/novosti/kultura',
 'https://izvmor.ru/novosti/nauka-i-obrazovanie',
 'https://izvmor.ru/novosti/novosti-kompaniy',
 'https://izvmor.ru/novosti/natsproekty',
 'https://izvmor.ru/novosti/obshchestvo/na-trasse-na-voditelyakh-zarabatyvayut-vse-spetskor-im-rasskazyvaet-o-tom-chto-mozhno-i-chto-nelzya-delat-dalnobojshchiku-v-rejse',
 'https://izvmor.ru/novosti/obshchestvo/na-trasse-na-voditelyakh-zarabatyvayut-vse-spetskor-im-rasskazyvaet-o-tom-chto-mozhno-i-chto-nelzya-delat-dalnobojshchiku-v-rejse',
 'https://izvmor.ru/novosti/obshchestvo',
 'https://izvmor.ru/novosti/proisshestviya/sledovateli-skr-proveryat-gibel-dvukh-zhitelej-mordovi

Разделим ссылки на новости и разделы с новостями. Для этого воспользуемся регулярными выражениями:

In [11]:
import re
pattern = 'https://izvmor.ru/novosti/[a-z-?=]+$'
sections = []
news = []

for u in full_urls:
    if re.match(pattern, u):
        if u not in sections:
            sections.append(u)
    else:
        if u not in news and u != 'https://izvmor.ru/novosti':
            if 'https://izvmor.ru/novosti?start=' not in u and 'https://izvmor.ru/novosti?layout=blog' not in u:
                news.append(u)  
news

['https://izvmor.ru/novosti/obshchestvo/na-trasse-na-voditelyakh-zarabatyvayut-vse-spetskor-im-rasskazyvaet-o-tom-chto-mozhno-i-chto-nelzya-delat-dalnobojshchiku-v-rejse',
 'https://izvmor.ru/novosti/proisshestviya/sledovateli-skr-proveryat-gibel-dvukh-zhitelej-mordovii-pri-pozhare',
 'https://izvmor.ru/novosti/obshchestvo/dietolog-kovalkov-rasskazal-o-boleznyakh-pri-kotorykh-rastet-zhivot',
 'https://izvmor.ru/novosti/nauka-i-obrazovanie/10-fevralya-devyatiklassnikam-mordovii-predstoit-itogovoe-sobesedovanie',
 'https://izvmor.ru/novosti/proisshestviya/stali-izvestny-podrobnosti-gibeli-lyudej-na-pozhare-v-romodanovskom-rajone',
 'https://izvmor.ru/novosti/proisshestviya/dva-cheloveka-pogibli-na-pozhare-v-mordovii',
 'https://izvmor.ru/novosti/obshchestvo/v-saranske-naschitali-75-prosrochennykh-razrytij',
 'https://izvmor.ru/novosti/obshchestvo/petr-tultaev-nazval-trevozhnoj-situatsiyu-s-podachej-otopleniya-v-doma-saranska',
 'https://izvmor.ru/novosti/obshchestvo/mer-moskvy-schitaet-n

Напишем функцию, которая вынимает из html-кода нужные нам данные:

In [12]:
def GetNews(url0):
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    sourse = soup0.find('meta', {'property' : 'og:site_name'}).attrs['content']
    date = soup0.find('meta', {'property': 'article:published_time'}).attrs['content']
    author = 'No author'
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
    text_list = soup0.find_all('p', {'class' : None})
    text = [t.text for t in text_list]
    final_text = ' '.join(text)
    final_text = final_text.replace('\n', ' ')
    cleaned_text = final_text.split('izv_mor@moris.ru ')[1].split(' Все права на материалы')[0]
    
    return url0, sourse, date, author, title, cleaned_text


In [13]:
news1 = GetNews(news[5])
news1

('https://izvmor.ru/novosti/proisshestviya/dva-cheloveka-pogibli-na-pozhare-v-mordovii',
 'Новостной портал Мордовии',
 '2020-10-19T08:29:22+03:00',
 'No author',
 'Два человека погибли на пожаре в Мордовии',
 ' В Мордовии два человека погибли при пожаре в частном доме. По данным регионального МЧС, трагедия произошла сегодня около четырех часов утра в совхозе «Садвинтрест» Ромодановского района. Дом отстоять не удалось. В ходе разбора сгоревших конструкций пожарные обнаружили тела двух мужчин 1977 и 1957 годов рождения. Причина пожара устанавливается. СУ СКР по РМ начал доследственная проверка.    ')

Создадим файл, в который будем вписывать данные про каждую новость, полученные с помощью функции выше:

In [14]:
with open('lab_news.txt', 'w', encoding='utf-8') as file:
    for n in news:
        res = GetNews(n)
        file.write('=====')
        file.write('\n')
        for i in res:
            file.write(i)
            file.write('\n')

Файл, который показывает, в какой секции находится новость:

In [15]:
with open('lab_news_graph.txt', 'w', encoding='utf-8') as file:
    for s in sections:
        for n in news:
            if n.startswith(s):
                file.write('===== \n')
                file.write(s)
                file.write('\n')
                res = GetNews(n)
                file.write('=====')
                file.write('\n')
                for i in res:
                    file.write(i)
                    file.write('\n')

Функции для создания двоичных деревьев:

In [11]:
def addValueToNode(node, value):
    if node[0] == value:
        return

    if value < node[0]:
        if node[1] == None:
            node[1] = [value, None, None]
        else:
            addValueToNode(node[1], value)

    else:
        if node[2] == None:
            node[2] = [value, None, None]
        else:
            addValueToNode(node[2], value)
            
    
def addValueToTree(root, value):
    if root == None or root == []:
        return [value, None, None]

    addValueToNode(root, value)
    return root

def printTreeDepth(node, right, left=0, strings=None, layer=0):
    if strings == None:
        strings = []

    if len(strings) <= layer:
        strings.append(' ' * left)

    elif len(strings[layer]) < left:
        strings[layer] += ' ' * (left - len(strings[layer]))

    dif1 = int((right - left - len(str(node[0]))) / 2) 
    dif2 = right - left - len(str(node[0])) - dif1

    strings[layer] += ' ' * dif1 + str(node[0]) + ' ' * dif2

    c = int((right + left) / 2)
    if node[1] != None:
        strings = printTreeDepth(node[1], c, left, strings, layer+1)
    if node[2] != None:
        strings = printTreeDepth(node[2], right, c+1, strings, layer+1)
        
    if layer == 0:
        for s in strings:
            print('\n', s)        
    return strings

def printTreeWidth2(nodes, width):
    if len(nodes) == 0:
        return

    woe = int(width / len(nodes))
    s = ''
    nodes2 = []
    for node in nodes:
        if node == None:
            s += ' ' * woe
            nodes2.extend([None, None])
        else:
            dif1 = int((woe - len(node[0])) / 2)
            dif2 = woe - len(node[0]) - dif1
            s += ' ' * dif1 + str(node[0]) + ' ' * dif2
            nodes2.append(node[1])
            nodes2.append(node[2])
    strings = [s]

    if any([n != None for n in nodes2]):
        strings.extend(printTreeWidth2(nodes2, width))
        
    return strings
    

def printTreeWidth(tree, width):
    strings = printTreeWidth2([tree], width)
    for s in strings:
        print('\n'+s)

printTree = printTreeDepth

Представим отсортированный текст каждой новости в виде двоичного дерева, добавим это дерево в общий список:

In [24]:
trees = []
for n in news:
    res = GetNews(n)
    text = res[5]
    words = text.split()
    sorted(words)
    tree = None
    for i in words:
        tree = addValueToTree(tree, i)
        trees.append(tree)


In [28]:
print(trees[0])
printTree(trees[0], 80)

['За', ['2020', ['(районными', ['(из', None, None], ['2019', ['178,', ['158,', ['111,', ['106.', None, None], None], None], None], None]], ['«Инцидент', ['7866', ['3234', ['301.', None, ['315,', None, None]], ['6870', ['6826;', ['387,', ['347,', None, None], ['5224.', None, ['588,', None, None]]], None], None]], ['VKontakte', ['Instagram', ['FB', None, None], None], None]], ['Для', ['В', ['Ардатовском', None, None], None], None]]], ['три', ['квартала', ['года', ['власти', ['Мордовии', ['Из', ['Зубово-Полянском', None, None], ['Ковылкинском', ['Инсарском', None, None], ['Лямбирском', None, None]]], ['администрациями,', ['Число', ['Наибольшее', None, ['Одноклассники', None, ['Твиттера', ['Саранск', ['Рузаевском', None, None], None], ['Темниковском', None, None]]]], None], ['ведомствами)', ['благодаря', ['аналитической', None, None], ['было', None, ['в', None, None]]], None]]], ['выросло,', ['втором', None, None], None]], ['государственной', ['городском', None, None], ['и', ['декабрь', No

['                                       За                                       ',
 '                  2020                                     три                  ',
 '     (районными           «Инцидент           квартала                –         ',
 '   (из       2019      7866       Для      года    представителями января  ',
 '           178,      3234 VKontakte В    властигосударственнойоргановсамоуправлениячисло',
 '           158,      301.6870InstagramАрдатовскомМордовиивыросло,городскомиместногопорассмотреносистеме',
 '           111,        315,6826;FB       Изадминистрациями,второмдекабрьинцидентовменеджмент»министерствамиотпришлосьссвязитретий',
 '           106.         387,             Зубово-ПолянскомКовылкинскомЧисловедомствами)запросовинцидента).кварталмаксимальноеместенихпервуюприходитрайонесетисравнения,третьем',
 '                        347,5224.         ИнсарскомЛямбирскомНаибольшееблагодаряжителейзафиксированоизинцидентов.количествонаокругеочередь,соцсетях',


Функция для индексации:

In [13]:
def bubbleSortIndex(a, N):
    index = list(range(N))
    for i in range(N-1):
        for j in range(N-i-1):
            if a[index[j]] > a[index[j+1]]:
                index[j], index[j+1] = index[j+1], index[j]
    return index

Сделаем список индексов для текста каждой новости:

In [14]:
indexes = []
for n in news:
    res = GetNews(n)
    text = res[5]
    words = text.split()
    sorted(words)
    index = bubbleSortIndex(words, len(words))
    indexes.append(index)

print(indexes[0])

[28, 13, 135, 130, 126, 122, 45, 3, 86, 117, 37, 112, 81, 103, 108, 75, 49, 26, 88, 84, 73, 21, 124, 104, 39, 0, 119, 87, 128, 110, 114, 12, 66, 94, 79, 105, 101, 90, 132, 51, 14, 19, 18, 24, 47, 55, 58, 61, 98, 109, 113, 118, 123, 127, 131, 17, 8, 77, 57, 4, 46, 99, 7, 44, 54, 52, 97, 9, 16, 89, 70, 38, 27, 96, 50, 93, 35, 2, 31, 67, 95, 65, 30, 23, 78, 10, 15, 33, 76, 82, 91, 29, 100, 6, 53, 60, 64, 59, 43, 5, 69, 32, 106, 115, 120, 133, 25, 48, 41, 63, 11, 62, 72, 20, 68, 71, 56, 40, 34, 83, 1, 92, 42, 22, 36, 74, 80, 85, 102, 107, 111, 116, 121, 125, 129, 134]
